In [1]:
#Code for Approach1: channel-to-channel reconstruction
import numpy as np
from scipy.io import loadmat
from sklearn.decomposition import  PCA
from sklearn.model_selection import LeaveOneOut
from ReClass import Subject, TSModel
import tensorflow as tf
import glob
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import scipy.stats as st
from tensorflow import keras
import gc
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session

Using TensorFlow backend.


In [0]:
#Create the structure of the model used for training and testing
#input includes 50 neurons, hidden layer includes 20 neurons, 
#and output includes 1 neuron
def createModel():  
    classifier = keras.models.Sequential()
    classifier.add(keras.layers.Dense(units = 20, kernel_initializer = 'normal', activation = 'relu',\
                         input_dim = 50))
    classifier.add(keras.layers.Dense(units = 1, activation = 'linear'))

    classifier.compile(optimizer='adadelta', loss='mean_squared_error')
    
    return classifier

#Train the model created above
def trainModel(x_train,y_train, scaler):

    print("Training....")
    classifier = createModel()
    x_train = scaler.fit_transform(x_train.transpose())
    y_train = scaler.fit_transform(y_train.reshape(-1, 1))
    classifier.fit(x_train, y_train, batch_size = y_train.size, epochs=20, verbose=0)
    return classifier       

#Split the subject's EEG data into training and testing sets
#using a 75/25 rule -> 75% for trainging, 25% for testing
def splitChannels(data):
  train = data[:int(len(data)*0.75)]
  test = data[int(len(data)*0.75):]
  return train,test

#Save the data in the matrix into a file
def saveMatrix(cof,index):
  with open("Sub-cof-%d.txt" % (index), "a+") as f:
    f.write("This is the correlation coefficient matrix for subject %d\n\n" % \
            (index))
    
    for i in cof:
      f.write("|")
      for j in i:
        f.write(" %f |" % (j))
      f.write("\n")

#Only use this function if there is access to a GPU   
def reset_keras():
  sess = get_session()
  clear_session()
  sess.close()
  sess = get_session()
  
  try:
    del classifier
  except:
    pass
  
  print(gc.collect())
  
  config = tf.ConfigProto()
  config.gpu_options.per_process_gpu_memory_fraction = 1
  config.gpu_options.visible_device_list = "0"
  set_session(tf.Session(config=config))
  
if __name__ == '__main__':
  
  subjects = [] #Creating a list of Subject classes to keep track of participant information
  
  #separate the .mat files needed from the directories
  subjectData = sorted(glob.glob('/content/drive/My Drive/Spark-2019/AD_MCI_Data/reformated data/resting data/extracted data 30chns/rest1 1min/rec*.mat'))#+ glob.glob('rec??.mat'))
  subjectData = [x[108:] for x in subjectData]
  
  #Loop through each file and save the subjects data, state, and channels in a Class Object
  for index,value in enumerate(subjectData):
      
      subjects.append(Subject(loadmat('/content/drive/My Drive/Spark-2019/AD_MCI_Data/reformated data/resting data/headerInfo'+value)['dx'][0],\
                              loadmat('/content/drive/My Drive/Spark-2019/AD_MCI_Data/reformated data/resting data/extracted data 30chns/rest1 1min'+value)['data'],\
                              index))

In [0]:
  models = [[None]*30]*30
  cof = np.ones((30,30))
  scaler = StandardScaler()
  
  #Loop through each subject and create a 30x30 causality matrix
  #Channel-to-channel reconstruction is used for each model
  #A right-shifting method is used on the EEG data to create enough input variables for the model
  #Reconstruct the EEG data, calculate the correlation coefficient
  #between the original and reconstructed data, and save the data in a file
  for s in subjects:
    
    for i,v1 in enumerate(s.data.transpose()):
      #reset_keras()
      train1,test1 = splitChannels(v1)
      tmp = train1

      for k in range(49):
        train1 = np.pad(train1,(1,0), mode='constant')[:-1]
        tmp = np.vstack((tmp,train1))
      train1 = tmp

      tmp = test1
      for k in range(49):
        test1 = np.pad(test1,(1,0), mode='constant')[:-1]
        tmp = np.vstack((tmp,test1))
      test1 = tmp

      for j,v2 in enumerate(s.data.transpose()):
          
          train2,test2 = splitChannels(v2)
          models[i][j] = trainModel(train1,train2, scaler)

          pred = models[i][j].predict(x=test1.transpose())
        
          if i != j:
            cof[i][j] = st.pearsonr(pred.flatten('C'),test2.transpose())[0]
      print(i)
    saveMatrix(cof,s.index)

  

W0710 15:21:24.419510 139907676432256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0710 15:21:24.422183 139907676432256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0710 15:21:24.424314 139907676432256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0710 15:21:24.450756 139907676432256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0710 15:21:24.452704 139907676432256 

22
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
0
31
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
1
640
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training....
Training...